# Introduction

This notebook shows how python with the aid of jupyter notebook can be used to perform particle size analysis. An example dataset from a sieving analysis is going to be used for this purpose. The data contains a set of given mesh sizes in μm and the mass of particles retained in each sieve in g. With this data, the particle distribution is going to be calculated and plotted with the following plots:- Histograms, cumulative distribution curve (undersize) and frequency distribution curve. 